In [1]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import os
import cv2
# import pickle
from tqdm import tqdm

In [2]:
def pic_augmentation(data,label,augmentation_num,batch_size):
    length=len(label)
    ultimate_list=[]
    list_of_pic_x=[]
    list_of_pic_y=[]
    for index in range(length):
        pic=data[index]
        y=label[index]
        for num in range(augmentation_num):
            Seed=(num,0)
            # pic=tf.image.stateless_random_brightness(pic,max_delta=0.3,seed=Seed)
            # pic=tf.image.stateless_random_contrast(pic,lower=0.1,upper=0.9,seed=Seed)
            pic=tf.image.stateless_random_flip_left_right(pic,seed=Seed)
            augmented_pic=tf.image.stateless_random_flip_up_down(pic,seed=Seed)
            # tf.image.stateless_random_jpeg_quality(pic,?)
            # tf.image.stateless_random_crop(pic,?)
            # tf.image.stateless_random_saturation(pic)
            # tf.image.stateless_random_hue(pic)
            # tf.image.stateless_sample_distorted_bounding_box(pic)
            data.append(augmented_pic)
            label.append(y)
    shuffled_data,shuffled_label=shuffle(data,label)
    for data,label in zip(shuffled_data,shuffled_label):
        list_of_pic_x.append(data)
        list_of_pic_y.append(label)
        if len(list_of_pic_x)==batch_size:
            ultimate_list.append((np.array(list_of_pic_x),np.array(list_of_pic_y)))
            list_of_pic_x=[]
            list_of_pic_y=[]
    return ultimate_list


def get_len(the_path):
    length=0
    categories=os.listdir(the_path)
    for category in categories:
        Path=os.path.join(the_path,category)
        length=length+len(os.listdir(Path))
    return length


# load and resizing the images
def get_image_big_batch(Batch,Big_Batch_size,the_path,Batch_size):
    categories=os.listdir(the_path)
    Start=Big_Batch_size*(Batch//((Big_Batch_size*len(categories))//Batch_size))
    categories=os.listdir(the_path)
    MyDataSet=[]
    for category in categories:
        label=categories.index(category)
        Path=os.path.join(the_path,category)
        if len(os.listdir(Path))>Start+Big_Batch_size:
            for pic_name in os.listdir(Path)[Start:Start+Big_Batch_size]:
                MyDataSet.append((os.path.join(Path,pic_name),label))
        else:
            for pic_name in os.listdir(Path)[Start:]:
                MyDataSet.append((os.path.join(Path,pic_name),label))
    shuffled_MyDataSet=shuffle(MyDataSet)
    return shuffled_MyDataSet


def get_portion_from_big_batch(batch,portion_size,the_big_batch,pic_size):
    Start=portion_size*(batch%(len(the_big_batch)//portion_size))
    X_portion_of_MyDataSet=[]
    Y_portion_of_MyDataSet=[]
    for path,Y in the_big_batch[Start:Start+portion_size]:
        try:
            image=cv2.imread(path)/255
            resized = cv2.resize(image, (pic_size, pic_size))
            X_portion_of_MyDataSet.append(resized)
            Y_portion_of_MyDataSet.append(Y)
        except:
            pass
        
    return X_portion_of_MyDataSet,Y_portion_of_MyDataSet


def train_on_batch(the_path,batch_size,portion_size,Big_Batch_size,n_epoch,pic_size,augmentation_num):
    loss_history=[]
    acc_history=[]
    categories=os.listdir(the_path)
    for epoch in range(n_epoch):
        for batch in tqdm(range((get_len(the_path))//batch_size)):
            if batch%(((Big_Batch_size*len(categories))//batch_size))==0:
                access_1='granted'
            else:
                access_1='not granted'
            if access_1=='granted':
                Big_Batch=get_image_big_batch(batch,Big_Batch_size,the_path,batch_size)   
            portion_x,portion_y=get_portion_from_big_batch(batch,portion_size,Big_Batch,pic_size)
            list_of_augmented_pic=pic_augmentation(portion_x,portion_y,augmentation_num,batch_size)
            for batch_x,batch_y in list_of_augmented_pic:
                loss,acc=model.train_on_batch(np.array(batch_x),np.array(batch_y))
                loss_history.append(loss)
                acc_history.append(acc)
        print('Epoch: %d ,     Train Loss %.4f,    Train Acc. %.4f' %
			(epoch+1, loss, acc))
    return {'loss_history':loss_history,'acc_history':acc_history}

In [13]:
__file__ = 'indoor.py'
dir_path = os.path.dirname(os.path.realpath(__file__))
# load and resizing the images
categories=os.listdir(os.path.join(dir_path,'test'))
MyDs=[]
for category in categories:
    label=categories.index(category)
    Path=os.path.join(os.path.join(dir_path,'test'),category)
    for pic_name in os.listdir(Path):
        try:
            image=cv2.imread(os.path.join(Path,pic_name))/255
            resized = cv2.resize(image, (310, 310))
            MyDs.append([resized,label])
        except:
            pass
        
X_MyDs=[]
Y_MyDs=[]
for x,y in MyDs:
    X_MyDs.append(x)
    Y_MyDs.append(y)
X_MyDs,Y_MyDs=shuffle(X_MyDs,Y_MyDs)


In [11]:
model=keras.applications.DenseNet201(include_top=False,input_shape=(310,310,3),classes=67)
for layer in model.layers[:-3]:
 layer.trainable=False
flatten=keras.layers.Flatten()(model.layers[-1].output)
dense1=keras.layers.Dense(1536,activation='elu')(flatten)
batchnorm1=keras.layers.BatchNormalization()(dense1)
dropout1=keras.layers.Dropout(0.5)(batchnorm1)
# dense2=keras.layers.Dense(1536,activation='elu')(dropout1)
# batchnorm2=keras.layers.BatchNormalization()(dense2)
# dropout2=keras.layers.Dropout(0.5)(batchnorm2)
# dense3=keras.layers.Dense(1536,activation='elu')(dropout2)
# batchnorm3=keras.layers.BatchNormalization()(dense3)
# dropout3=keras.layers.Dropout(0.5)(batchnorm3)
# dense4=keras.layers.Dense(1536,activation='elu')(dropout3)
# batchnorm4=keras.layers.BatchNormalization()(dense4)
# dropout4=keras.layers.Dropout(0.5)(batchnorm4)
# dense5=keras.layers.Dense(1536,activation='elu')(dropout4)
# batchnorm5=keras.layers.BatchNormalization()(dense5)
# dropout5=keras.layers.Dropout(0.5)(batchnorm5)
dense6=keras.layers.Dense(67,activation='softmax')(dropout1)

model=keras.Model(inputs=model.inputs,outputs=dense6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 310, 310, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 316, 316, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 155, 155, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [12]:
path='D:\datasets\indoorCVPR\indoorCVPR_09\\train'
history=train_on_batch(the_path=path,batch_size=32,portion_size=32,Big_Batch_size=512,n_epoch=10,pic_size=310,augmentation_num=2)

100%|██████████| 167/167 [05:09<00:00,  1.85s/it]


Epoch: 1 ,     Train Loss 1.4058,    Train Acc. 0.6250


100%|██████████| 167/167 [04:46<00:00,  1.71s/it]


Epoch: 2 ,     Train Loss 0.2633,    Train Acc. 0.9375


100%|██████████| 167/167 [04:45<00:00,  1.71s/it]


...

Epoch: 8 ,     Train Loss 0.0130,    Train Acc. 1.0000


100%|██████████| 167/167 [04:45<00:00,  1.71s/it]


Epoch: 9 ,     Train Loss 0.0317,    Train Acc. 1.0000


100%|██████████| 167/167 [04:45<00:00,  1.71s/it]

Epoch: 10 ,     Train Loss 0.0932,    Train Acc. 0.9688


In [2]:
model.save('model.h5') #for saving the best model yet
#model.save('model2.h5') # for saving model thats gonna change cuz we do fine tuning

# model=keras.models.load_model('model.h5')

In [18]:
prediction=model.predict(np.array(X_MyDs))

42/42 [==============================] - 8s 188ms/step


In [19]:
prediction2=np.argmax(prediction,axis=1)

In [21]:
print(f'accuracy_score:{accuracy_score(Y_MyDs,prediction2)}')

accuracy_score:0.7098950524737632
